# Языковые модели

Какое слово в последовательности вероятнее: 

Поезд прибыл на
* вокзал
* север

Какая последовательность вероятнее:
* Вокзал прибыл поезд на
* Поезд прибыл на вокзал

Языковая модель [language model, LM]  позволяет оценить вероятность следующего слова в последовательности  $P(w_n | w_1, \ldots, w_{n-1})$ и оценить вероятность всей последовательности слов $P(w_1, \ldots, w_n)$.

### Приложения:
* Задачи, в которых нужно обработать сложный и зашумленный вход: распознавание речи, распознавание сканированных и рукописных текстов;
* Исправление опечаток
* Машинный перевод
* Подсказка при наборе 

### Виды моделей:
* Счетные модели
    * цепи Маркова
* Нейронные модели, обычно реккурентные нейронные сети с LSTM/GRU
* seq2seq архитектуры

## Модель $n$-грам

Пусть $w_{1:n}=w_1,\ldots,w_m$ – последовательность слов.

Цепное правило:
$ P(w_{1:m}) = P(w_1) P(w_2 | w_1) P(w_3 | w_{1:2}) \ldots P(w_m | w_{1:m-1}) = \prod_{k=1}^{m} P(w_k | w_{1:k-1}) $

Но оценить $P(w_k | w_{1:k-1})$ не легче! 

Переходим к $n$-грамам: $P(w_{i+1} | w_{1:i}) \approx P(w_{i+1} | w_{i-n:i})  $ , то есть, учитываем $n-1$ предыдущее слово. 

Модель
* униграм: $P(w_k)$
* биграм: $P(w_k | w_{k-1})$
* триграм: $P(w_k | w_{k-1} w_{k-2})$


Т.е. используем Марковские допущения о длине запоминаемой цепочки.

* Вероятность следующего слова в последовательности: $ P(w_{i+1} | w_{1:i}) \approx P(w_{i-n:i}) $
* Вероятность всей последовательности слов: $P(w_{1:n}) = \prod_{k=1}^l P(w_k | w_{k-n+1: k-1}) $

### Качество модели  $n$-грам

Перплексия: насколько хорошо модель предсказывает выборку. Чем ниже значение перплексии, тем лучше.

$PP(\texttt{LM}) = 2 ^ {-\frac{1}{m} \log_2 \texttt{LM} (w_i | w_{1:i-1})}$

## Счетные языковые модели

### ММП оценки вероятностей
$ P_{MLE}(w_k | w_{k-n+1:k-1}) = \frac{\texttt{count}(w_{k-n+1:k-1} w_k )}{\texttt{count}(w_{k-n+1:k-1} )} $

В модели биграм:

$ P_{MLE}(w_k | w_{k-1}) = \frac{\texttt{count}(w_{k-1} w_k )}{\texttt{count}(w_{k-1} )} $

Возникает проблема нулевых вероятностей!

### Аддитивное сглаживание Лапласа

$ P(w_k | w_{k-1}) = \frac{\texttt{count}(w_{k-1} w_k ) + \alpha}{\texttt{count}(w_{k-1} ) + \alpha |V|} $

## Модели биграм в NLTK

In [1]:
import nltk
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, TimeDistributed, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout

In [2]:
names = [name.strip().lower() for name in 
         open('C:/Users/adwiz/Documents/Courses/datascience_netology/datasets/nlp/dinos.txt').readlines()]
print(names[:10])

['aachenosaurus', 'aardonyx', 'abdallahsaurus', 'abelisaurus', 'abrictosaurus', 'abrosaurus', 'abydosaurus', 'acanthopholis', 'achelousaurus', 'acheroraptor']


In [3]:
chars = [char for name in names for char in name]
freq = nltk.FreqDist(chars)
print(list(freq.keys()))

['a', 'c', 'h', 'e', 'n', 'o', 's', 'u', 'r', 'd', 'y', 'x', 'b', 'l', 'i', 't', 'p', 'v', 'm', 'g', 'f', 'j', 'k', 'w', 'z', 'q']


In [4]:
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(chars))
cfreq['a']

FreqDist({'u': 792, 'n': 354, 't': 213, 's': 187, 'l': 146, 'r': 131, 'c': 109, 'p': 96, 'm': 74, 'e': 48, ...})

In [5]:
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)
print('p(a a) = %1.4f' %cprob['a'].prob('a'))
print('p(a b) = %1.4f' %cprob['a'].prob('b'))
print('p(a u) = %1.4f' %cprob['a'].prob('u'))

p(a a) = 0.0105
p(a b) = 0.0129
p(a u) = 0.3185


In [6]:
from math import log
log(cprob['a'].prob('a')) + log(cprob['a'].prob('b')) + log(cprob['a'].prob('c'))

-12.041317008359863

In [7]:
l = sum([freq[char] for char in freq])
def unigram_prob(char):
    return freq[char] / l
print(f'p(a) = {unigram_prob("a"):1.4f}')
# print('p(a) = %1.4f' %unigram_prob('a'))

p(a) = 0.1354


In [8]:
[bi for bi in nltk.bigrams('aachenosaurus')]

[('a', 'a'),
 ('a', 'c'),
 ('c', 'h'),
 ('h', 'e'),
 ('e', 'n'),
 ('n', 'o'),
 ('o', 's'),
 ('s', 'a'),
 ('a', 'u'),
 ('u', 'r'),
 ('r', 'u'),
 ('u', 's')]

In [9]:
cprob["a"].generate()

'u'

#### Задание 1

1. Напишите функцию для оценки вероятности имени динозавра. 
2. Найдите наиболее вероятное имя динозавра из данного списка. 

In [10]:
def name_prob(name):
    p = unigram_prob(name[0])
    for i in range(len(name) - 1):
        p *= cprob[name[i]].prob(name[i+1])
    return p

In [11]:
name_prob(names[0])

2.0222358416238476e-10

In [12]:
names[0]

'aachenosaurus'

In [13]:
def generate_name(cprob, first_char, num_chars):
    name = ''
    name += first_char
    for i in range(num_chars):
        char = cprob[first_char].generate()
        name += char
        first_char = char
    return name

generate_name(cprob, 't', 9)

'tesangowur'

## Нейронные языковые модели

* Вход: $n$-грамы $w_{1:k}$
* $v(w_i)$ – эмбеддинг слова $w_i$, $v(w_i) \in \mathbb{R}^{d_{emb}}$, $d_{emb}$ – размерность эмбеддинга, $v(w) = E_{[w]}$
* $x = [v(w_1), v(w_2), \ldots , v(w_k)]$

$\widehat{y} = P(w_i | w_{1:k} ) = \texttt{LM}(w_{1:k}) = \texttt{softmax}(hW^2 +b^2)$

$h = g(xW^1+b^1)$

$w_i \in V$, $E \in \mathbb{R}^{|V|\times d_{emb}}, W^1 \in \mathbb{R}^{k \cdot d_{emb} \times d_{hid}}, b^1 \in \mathbb{R} ^ {d_{hid}}, W^2 \in \mathbb{R}^{d_{hid} \times |V|}, b^2 \in \mathbb{R} ^ {|V|}$

### Семплирование в нейронных языковых моделях 
### (Генерация текстов с помощью нейронных языковых моделей)

1. Задать начальную последовательность символов длины $k$ (/слов)
2. Предсказать распределение вероятностей слов с условием на $k$ предыдущих слов
3. 1. Выбрать слово с наибольшей вероятностью
3. 2. Выбрать слово по предсказаному распределению
4. Сдвинуть окно на одно слово и повторить 

#### Линейный поиск  (beam search)
Всегда помним $h$ наиболее вероятных гипотез:
1. Для генерации первого слова в последоватительности генерируем $h$ кандидатов, а не 1
2. Генерируем $h \times h$ кандидатов для второго слова и храним только $h$ наиболее вероятных


In [14]:
alphabet = list(set(chars))
print('total chars:', len(alphabet))

total chars: 26


In [15]:
maxlen = 5
step = 1
ngrams = []
next_chars = []
for name in names:
    for i in range(0, len(name) - maxlen, step):
        ngrams.append(' '.join([char for char in name[i: i + maxlen]]))
        next_chars.append(name[i + maxlen])
print('nb ngrams:', len(ngrams))
print(ngrams[0], next_chars[0])
print(ngrams[1], next_chars[1])

nb ngrams: 10701
a a c h e n
a c h e n o


In [16]:
tokenizer = Tokenizer(num_words=len(alphabet))
tokenizer.fit_on_texts(ngrams)

sequences = tokenizer.texts_to_sequences(ngrams)
X_train = pad_sequences(sequences, maxlen=maxlen)
sequences = tokenizer.texts_to_sequences(next_chars)
y_train = tokenizer.sequences_to_matrix(sequences)
X_train[0]

array([ 1,  1, 12, 11,  7])

In [17]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [18]:
char_index = tokenizer.word_index
index_char = {i: c for c, i in char_index.items()}

In [19]:
model = Sequential()
model.add(Embedding(len(alphabet), 50, input_length=maxlen))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(len(alphabet), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=2, verbose=0)

In [21]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) # temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.choice(range(len(alphabet)), p=preds)
    return probas

In [22]:
import random
import sys
import numpy as np

generated = ''
seed = 'aleks'
generated += seed
print('----- Generating with seed: "' + seed + '"')
print(generated)

for i in range(8):
    sequences = tokenizer.texts_to_sequences([' '.join([char for char in generated[-maxlen:]])])
    X_pred = pad_sequences(sequences, maxlen=maxlen)
    preds = model.predict(X_pred, verbose=0)[0]
    next_index = sample(preds)
    next_char = index_char[next_index]
    generated += next_char
    print(generated)

----- Generating with seed: "aleks"
aleks
aleksa
aleksau
aleksaur
aleksauru
aleksaurus
aleksaurusa
aleksaurusau
aleksaurusaur


### Рекуррентные нейронные языковые модели

RNN позволяют уйти от Марковских допущений и позволяют учитывать предысторию произвольной длины.

$x_{1:n} = x_1, x_2, \ldots, x_n$, $x_i \in \mathbb{R}^{d_{in}}$

$y_n = RNN(x_{1:n})$, $y_n \in \mathbb{R}^{d_{out}}$

Для каждого префикса $x_{i:i}$ $y_i$ – выходной вектор.

$y_i = RNN(x_{1:i})$

$y_{1:n} = RNN^{*}(x_{1:n})$, $y_i \in \mathbb{R}^{d_{out}}$

#### Управляемые архитектуры

RNN трудно обучать: проблема исчезающего градиента. Уйти от нее помогают управляемые нейроны специального вида: LSTM и GRU.

$s_i$ – память нейронной сети. Каждое использование $R$ считывает и видоизменяет всю память. 

Управляемый доступ к памяти: $g \in {0,1}^n$:

$s_{i+1} \leftarrow g \odot x + (1-g) \odot s_{i}$

Дифференцируемое управление:

$g \in \mathbb{R}^n $:

$s_{i+1} \leftarrow \sigma(g) \odot x + (1-g) \odot s_{i}$  

#### Long short-term memory

$s_j = R_{LSTM}(s_{j-1}, x_j) = [c_j, h_j]$,  $c_j$ – вектор памяти, $h_j$ – скрытое состояние

$c_j = f \odot c_{j-1} + i \odot z_j$ – обновление памяти 

$h_j = o \odot tanh(c_j)$

$i = \sigma(x_j W^{xi} + h_{j-1} W^{hi}) $ – что нужно выбрать из входа ($i \odot z$)

$f = \sigma(x_j W^{xf} + h_{j-1} W^{hf}) $ – что нужно забыть ($f \odot c_{j-1}$)

$o = \sigma(x_j W^{xo} + h_{j-1} W^{ho}) $– что нужно выдать на выход ($o \odot c_{j}$)

$z = tanh(x_j W^{xz} + h_{j-1} W^{hz})$ – обновление 

$y_j = O_{LSTM}(s_j) = h_j$ – выход 




$x_i \in \mathbb{R}^{d_{in}}$, $y_i \in \mathbb{R}^{d_{out}}$, $s_i \in \mathbb{R}^{2 \times d_{out}}$

$c_j, h_j, i, f, o, z \in \mathbb{R}^{ d_{out}}$

$W^{x.} \in \mathbb{R}^{d_{in} \times d_{out}}$, $W^{h.} \in \mathbb{R}^{d_{out} \times d_{out}}$

http://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [23]:
from tensorflow.keras.utils import to_categorical
import numpy as np

X_names = ['bos ' + ' '.join(name) for name in names]
Y_names = [' '.join(name) + ' eos' for name in names]
maxlen = max([len(name) for name in names]) + 1

In [27]:
X_names[:100]

['bos a a c h e n o s a u r u s',
 'bos a a r d o n y x',
 'bos a b d a l l a h s a u r u s',
 'bos a b e l i s a u r u s',
 'bos a b r i c t o s a u r u s',
 'bos a b r o s a u r u s',
 'bos a b y d o s a u r u s',
 'bos a c a n t h o p h o l i s',
 'bos a c h e l o u s a u r u s',
 'bos a c h e r o r a p t o r',
 'bos a c h i l l e s a u r u s',
 'bos a c h i l l o b a t o r',
 'bos a c r i s t a v u s',
 'bos a c r o c a n t h o s a u r u s',
 'bos a c r o t h o l u s',
 'bos a c t i o s a u r u s',
 'bos a d a m a n t i s a u r u s',
 'bos a d a s a u r u s',
 'bos a d e l o l o p h u s',
 'bos a d e o p a p p o s a u r u s',
 'bos a e g y p t o s a u r u s',
 'bos a e o l o s a u r u s',
 'bos a e p i s a u r u s',
 'bos a e p y o r n i t h o m i m u s',
 'bos a e r o s t e o n',
 'bos a e t o n y x a f r o m i m u s',
 'bos a f r o v e n a t o r',
 'bos a g a t h a u m a s',
 'bos a g g i o s a u r u s',
 'bos a g i l i s a u r u s',
 'bos a g n o s p h i t y s',
 'bos a g r o s 

In [28]:
Y_names[:100]

['a a c h e n o s a u r u s eos',
 'a a r d o n y x eos',
 'a b d a l l a h s a u r u s eos',
 'a b e l i s a u r u s eos',
 'a b r i c t o s a u r u s eos',
 'a b r o s a u r u s eos',
 'a b y d o s a u r u s eos',
 'a c a n t h o p h o l i s eos',
 'a c h e l o u s a u r u s eos',
 'a c h e r o r a p t o r eos',
 'a c h i l l e s a u r u s eos',
 'a c h i l l o b a t o r eos',
 'a c r i s t a v u s eos',
 'a c r o c a n t h o s a u r u s eos',
 'a c r o t h o l u s eos',
 'a c t i o s a u r u s eos',
 'a d a m a n t i s a u r u s eos',
 'a d a s a u r u s eos',
 'a d e l o l o p h u s eos',
 'a d e o p a p p o s a u r u s eos',
 'a e g y p t o s a u r u s eos',
 'a e o l o s a u r u s eos',
 'a e p i s a u r u s eos',
 'a e p y o r n i t h o m i m u s eos',
 'a e r o s t e o n eos',
 'a e t o n y x a f r o m i m u s eos',
 'a f r o v e n a t o r eos',
 'a g a t h a u m a s eos',
 'a g g i o s a u r u s eos',
 'a g i l i s a u r u s eos',
 'a g n o s p h i t y s eos',
 'a g r o s a u 

In [29]:
X_names[0], Y_names[0]

('bos a a c h e n o s a u r u s', 'a a c h e n o s a u r u s eos')

In [24]:
tokenizer = Tokenizer(num_words=len(alphabet)+2)
tokenizer.fit_on_texts(X_names + Y_names)

sequences = tokenizer.texts_to_sequences(X_names)
X_train = pad_sequences(sequences, maxlen=maxlen, padding='post')

sequences = tokenizer.texts_to_sequences(Y_names)
Y_train = pad_sequences(sequences, padding='post')

Y_train_cat = [to_categorical(sent, num_classes=len(alphabet) + 2) for sent in Y_train]
Y_train = np.asarray(Y_train_cat)

In [31]:
X_train[0]

array([10,  1,  1, 15, 14,  8,  6,  4,  2,  1,  3,  5,  3,  2,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [32]:
Y_train[0]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0

In [34]:
X_train.shape, Y_train.shape

((1536, 27), (1536, 27, 28))

In [25]:
print(X_names[0])
print(Y_names[0])


print(X_train.shape)
print(Y_train.shape)

print(tokenizer.word_index['bos'])
print(tokenizer.word_index['eos'])

bos a a c h e n o s a u r u s
a a c h e n o s a u r u s eos
(1536, 27)
(1536, 27, 28)
10
11


In [26]:
char_index = tokenizer.word_index
index_char = {i: c for c, i in char_index.items()}

In [36]:
model = Sequential()

model.add(Embedding(len(alphabet) + 2, 30, input_length=maxlen))
model.add(LSTM(128, return_sequences = True))

model.add(Dense(len(alphabet) +2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
for iteration in range(1, 20):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, Y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose=1)

1/1 [==============================] - 0s 466ms/step - loss: 1.5764 - accuracy: 0.5221


In [46]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) #/ temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.choice(range(len(alphabet)+2), p = preds)
    return probas

In [48]:
generated = ''
seed = 'bos'
generated += seed + ' '
print('----- Generating with seed: "' + seed + '"')
print(generated)

for i in range(7):
    sequences = tokenizer.texts_to_sequences([seed])
    X_pred = pad_sequences(sequences, maxlen=maxlen, padding= 'post')
    
    preds = model.predict(X_pred, verbose=0)[0]
    samples = [sample(p) for p in preds]
    next_index = samples[i]
    while next_index == 0 or next_index == 10:
        samples = [sample(p) for p in preds]
        next_index = samples[i]
    next_char = index_char[next_index]
    generated += next_char + ' '
    print(generated)
    seed += next_char
    if next_char == 'eos':
        break

----- Generating with seed: "bos"
bos 
bos m 
bos m s 
bos m s z 
bos m s z d 
bos m s z d s 
bos m s z d s b 
bos m s z d s b l 
